# <center>Алгоритм Adaboost</center>

In [1]:
import numpy as np
import pandas as pd

In [11]:
data = pd.read_csv(r'C:\Users\Lenovo\Documents\data_adaboost.csv', header=None)
data.head()

,0,1,2
0,1.3,4.0,1
1,1.0,2.0,1
2,2.0,2.5,1
3,2.0,3.5,-1
4,2.5,0.5,1


In [12]:
X = data.drop(2,axis=1)
X.head()

,0,1
0,1.3,4.0
1,1.0,2.0
2,2.0,2.5
3,2.0,3.5
4,2.5,0.5


In [14]:
y = data[2]
y.head()

0    1
1    1
2    1
3   -1
4    1
Name: 2, dtype: int64

**Задание.** Реализуйте алгоритм Adaboost.

В качестве базового алгоритма возьмем `DecisionTreeClassifier` глубины 1. Количество базовых алгоритмов $T=3$.

- В переменной `self.sample_weight` будем хранить текущие значения весов объектов.
- В методе `fit` последовательно будет производиться построение пней решения. На первом шаге алгоритма веса всех объектов одинаковы, т.е.:
$$w_i^0=\frac{1}{l}, i=1,\ldots,l,$$
где $l$ &ndash; количество объектов обучающей выборки.
- На каждом шаге алгоритма необходимо обучить пень решения с текущими весами объектов. Веса объектов задаются в методе `fit(X, y, sample_weight=...)` пня.
- После того, как пень обучен, вычислить:
$$\epsilon_t=\sum_{i=1}^{l} w_i[y_i \neq b_t(x_i)],$$
$$\alpha_t=\frac{1}{2}\log\frac{1-\epsilon_t}{\epsilon_t}.$$
- И обновить веса объектов:
$$w_i^t=w_i^{t-1}\exp(-\alpha_t y_i b_t(x_i)), i=1,\ldots,l,$$
$$w_i^t=\frac{w_i^t}{\sum_{j=1}^{l} w_j^t}.$$

- Обученные пни будем хранить в переменной `self.trees`.

1. Какими оказались веса объектов после обучения?
2. Пользуясь полученной композицией алгоритмов, классифицируйте точку (4,4) (метод `predict` в реализации).
3. За сколько шагов алгоритм классифицирует все точки верно, если глубина дерева равна 2?

In [73]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode

class AdaboostClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=3, max_depth=1, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []
        self.sample_weight = []
        
    def fit(self, X, y):
        n = len(y)
        self.sample_weight = [1/n for _ in range(n)]
        for _ in range(self.n_estimators):
            tree = DecisionTreeClassifier(max_depth=self.max_depth,random_state=self.random_state)
            tree.fit(X,y,self.sample_weight)
            y_pred = tree.predict(X)
            self.sample_weight = self.new_weights(self.sample_weight,y,y_pred)
            self.trees.append(tree)

        
    def predict(self, X):
        res = []
        for t in self.trees:
            res.append(t.predict(X).tolist())
        return pd.DataFrame(res).apply(lambda x: mode(x).mode,axis=0)

    @staticmethod
    def new_weights(w,y,b):
        mask = list(y!=b)
        w = pd.Series(w)
        eps = sum(w[mask])

        n_w = []
        alpha = np.log((1-eps)/eps)/2
        for i in range(len(w)):
            n_w.append(w[i]*np.exp(-alpha*y[i]*b[i]))
        return n_w/sum(n_w)




In [74]:
clf = AdaboostClassifierCustom()

In [75]:
clf.fit(X,y)

In [76]:
clf.sample_weight

array([0.07291667, 0.02083333, 0.125     , 0.0625    , 0.125     ,
       0.125     , 0.0625    , 0.125     , 0.0625    , 0.07291667,
       0.07291667, 0.07291667])

In [81]:
clf.predict([[4,4]])

,0
0,-1


In [101]:
from sklearn.metrics import accuracy_score

In [87]:
params= {'n_estimators':[1,2,3,4,5,6]}

In [110]:
for i in range(1,10):
    clf = AdaboostClassifierCustom(max_depth=2,n_estimators=i)
    clf.fit(X,y)
    y_pred = clf.predict(X).values.tolist()[0]
    print(accuracy_score(y.tolist(),y_pred))


0.9166666666666666
0.9166666666666666
1.0
1.0
1.0
1.0
1.0
1.0
1.0


3 шага